<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=46ba8dde177d91cff691847b701b5999f3d20a77a0cb0b29b598945aa2c11e3e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3711 sha256=c76136593b2cb35c64cd9ea15352d6c4860381392ea605863d59f0a54e74060e
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, min_price=100, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4708]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
経過時間: 381.9783453941345秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,2528100,770.700012
1,三井松島ホールディングス,1518,鉱業,792000,2822.000000
2,ＩＮＰＥＸ,1605,鉱業,41718300,2116.000000
3,石油資源開発,1662,鉱業,919400,4905.000000
4,安藤・間,1719,建設業,700100,1156.000000
...,...,...,...,...,...
407,エヌ・ティ・ティ・データ,9613,情報・通信業,4960800,1864.000000
408,ＳＣＳＫ,9719,情報・通信業,525100,2408.500000
409,トラスコ中山,9830,卸売業,779900,2339.000000
410,ヤマダホールディングス,9831,小売業,3894700,449.799988


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,2528100,770.700012
1,ＩＮＰＥＸ,1605,鉱業,41718300,2116.000000
2,石油資源開発,1662,鉱業,919400,4905.000000
3,安藤・間,1719,建設業,700100,1156.000000
4,コムシスホールディングス,1721,建設業,802400,3121.000000
...,...,...,...,...,...
271,電源開発,9513,電気・ガス業,708500,2248.000000
272,東京瓦斯,9531,電気・ガス業,1069500,3358.000000
273,エイチ・アイ・エス,9603,サービス業,2541400,2130.000000
274,エヌ・ティ・ティ・データ,9613,情報・通信業,4960800,1864.000000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,2528100,770.700012
1,ＩＮＰＥＸ,1605,鉱業,41718300,2116.000000
2,石油資源開発,1662,鉱業,919400,4905.000000
3,安藤・間,1719,建設業,700100,1156.000000
4,コムシスホールディングス,1721,建設業,802400,3121.000000
...,...,...,...,...,...
163,北海道電力,9509,電気・ガス業,1615400,613.000000
164,電源開発,9513,電気・ガス業,708500,2248.000000
165,東京瓦斯,9531,電気・ガス業,1069500,3358.000000
166,エイチ・アイ・エス,9603,サービス業,2541400,2130.000000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,三井松島ホールディングス,1518,鉱業,792000,2822.000000
1,オリエンタル白石,1786,建設業,584200,313.000000
2,三井住友建設,1821,建設業,4736300,392.000000
3,ＭＩＸＩ,2121,サービス業,502700,2494.000000
4,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,4620800,723.299988
...,...,...,...,...,...
133,大阪瓦斯,9532,電気・ガス業,1123300,2289.500000
134,エヌ・ティ・ティ・データ,9613,情報・通信業,4960800,1864.000000
135,トラスコ中山,9830,卸売業,779900,2339.000000
136,ヤマダホールディングス,9831,小売業,3894700,449.799988


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ＭＩＸＩ,2121,サービス業,502700,2494.000000
1,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,4620800,723.299988
2,リンクアンドモチベーション,2170,サービス業,605100,417.000000
3,カルビー,2229,食料品,584900,2731.000000
4,システナ,2317,情報・通信業,2016900,251.000000
...,...,...,...,...,...
78,ＫＤＤＩ,9433,情報・通信業,5102800,4177.000000
79,イーレックス,9517,電気・ガス業,600800,1086.000000
80,レノバ,9519,電気・ガス業,1676300,1494.000000
81,エヌ・ティ・ティ・データ,9613,情報・通信業,4960800,1864.000000


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 100円以上、5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_standard

Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6032]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,250400,2583.0
1,住石ホールディングス,1514,鉱業,739900,352.0
2,塩水港精糖,2112,食料品,366400,238.0
3,クルーズ,2138,情報・通信業,3017400,1160.0
4,ｎｍｓ ホールディングス,2162,サービス業,203000,424.0
...,...,...,...,...,...
90,乾汽船,9308,海運業,282300,1404.0
91,日本テレホン,9425,情報・通信業,227800,384.0
92,インプレスホールディングス,9479,情報・通信業,227100,203.0
93,ＫＮＴ－ＣＴホールディングス,9726,サービス業,776200,1603.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,塩水港精糖,2112,食料品,366400,238.000000
1,極楽湯ホールディングス,2340,サービス業,231300,418.000000
2,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,335000,676.000000
3,出前館,2484,情報・通信業,544400,439.000000
4,ＳＡＮＫＯ ＭＡＲＫＥＴＩＮＧ ＦＯＯＤＳ,2762,小売業,273800,191.000000
5,アップルインターナショナル,2788,卸売業,737400,423.000000
6,焼津水産化学工業,2812,食料品,566000,1215.000000
7,ポラリス・ホールディングス,3010,サービス業,652500,178.000000
8,デリカフーズホールディングス,3392,卸売業,247600,642.000000
9,山喜,3598,繊維製品,287500,163.000000


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,塩水港精糖,2112,食料品,366400,238.0
1,極楽湯ホールディングス,2340,サービス業,231300,418.0
2,アップルインターナショナル,2788,卸売業,737400,423.0
3,焼津水産化学工業,2812,食料品,566000,1215.0
4,ポラリス・ホールディングス,3010,サービス業,652500,178.0
5,デリカフーズホールディングス,3392,卸売業,247600,642.0
6,山喜,3598,繊維製品,287500,163.0
7,クラウディアホールディングス,3607,繊維製品,569200,909.0
8,ジェイ・エスコムホールディングス,3779,情報・通信業,4802100,164.0
9,パス,3840,情報・通信業,365800,109.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,250400,2583.0
1,住石ホールディングス,1514,鉱業,739900,352.0
2,クルーズ,2138,情報・通信業,3017400,1160.0
3,ｎｍｓ ホールディングス,2162,サービス業,203000,424.0
4,クシム,2345,情報・通信業,207200,308.0
5,セリア,2782,小売業,234500,2270.0
6,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,309700,1125.0
7,プロパスト,3236,不動産業,434900,144.0
8,トラスト,3347,小売業,280700,267.0
9,セルシス,3663,情報・通信業,224500,680.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,739900,352.0
1,クシム,2345,情報・通信業,207200,308.0
2,プロパスト,3236,不動産業,434900,144.0
3,セルシス,3663,情報・通信業,224500,680.0
4,ｅｎｉｓｈ,3667,情報・通信業,520600,308.0
5,ＫｅｙＨｏｌｄｅｒ,4712,サービス業,230100,976.0
6,シダックス,4837,サービス業,278300,561.0
7,ハーモニック・ドライブ・システムズ,6324,機械,843600,4145.0
8,シライ電子工業,6658,電気機器,600100,511.0
9,ヘリオス テクノ ホールディング,6927,電気機器,796300,430.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_grows

Not Found: No data found, symbol may be delisted
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,236800,913.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,227900,720.0
2,ジーエヌアイグループ,2160,医薬品,688700,1343.0
3,ウェッジホールディングス,2388,その他金融業,922800,146.0
4,ユナイテッド,2497,サービス業,230300,899.0
...,...,...,...,...,...
91,トリドリ,9337,サービス業,521000,3015.0
92,ＧＥＮＯＶＡ,9341,サービス業,320500,1712.0
93,リニューアブル・ジャパン,9522,電気・ガス業,1277400,1063.0
94,Ｍ＆Ａ総研ホールディングス,9552,サービス業,476900,3365.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,227900,720.0
1,ジーエヌアイグループ,2160,医薬品,688700,1343.0
2,ウェッジホールディングス,2388,その他金融業,922800,146.0
3,ユナイテッド,2497,サービス業,230300,899.0
4,ベースフード,2936,食料品,1058300,581.0
5,タスキ,2987,不動産業,234400,1093.0
6,海帆,3133,小売業,1537700,477.0
7,ロコンド,3558,小売業,393200,1875.0
8,ディジタルメディアプロフェッショナル,3652,情報・通信業,218100,3200.0
9,アプリックス,3727,情報・通信業,783600,168.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,688700,1343.0
1,ユナイテッド,2497,サービス業,230300,899.0
2,ベースフード,2936,食料品,1058300,581.0
3,タスキ,2987,不動産業,234400,1093.0
4,アプリックス,3727,情報・通信業,783600,168.0
5,Ａｉｍｉｎｇ,3911,情報・通信業,4525900,549.0
6,アララ,4015,情報・通信業,426400,630.0
7,ヤプリ,4168,情報・通信業,289600,1302.0
8,Ｐｈｏｔｏｓｙｎｔｈ,4379,情報・通信業,211500,496.0
9,ＢＡＳＥ,4477,情報・通信業,17455900,361.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,236800,913.0
1,サンクゼール,2937,食料品,698500,3235.0
2,サンワカンパニー,3187,小売業,219400,606.0
3,スタジオアタオ,3550,小売業,734700,225.0
4,アドバンスト・メディア,3773,情報・通信業,203200,1623.0
5,ドリコム,3793,情報・通信業,378000,489.0
6,クラウドワークス,3900,情報・通信業,214400,1376.0
7,プラスアルファ・コンサルティング,4071,情報・通信業,253800,2720.0
8,ココナラ,4176,情報・通信業,885200,314.0
9,バンク・オブ・イノベーション,4393,情報・通信業,388800,5000.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,アドバンスト・メディア,3773,情報・通信業,203200,1623.0
1,ドリコム,3793,情報・通信業,378000,489.0
2,クラウドワークス,3900,情報・通信業,214400,1376.0
3,プラスアルファ・コンサルティング,4071,情報・通信業,253800,2720.0
4,ココナラ,4176,情報・通信業,885200,314.0
5,バンク・オブ・イノベーション,4393,情報・通信業,388800,5000.0
6,アクリート,4395,情報・通信業,262800,1228.0
7,ＪＤＳＣ,4418,情報・通信業,207600,863.0
8,アンジェス,4563,医薬品,394900,101.0
9,キャンバス,4575,医薬品,387000,995.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_standard_2

Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6032]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,739900,352.000000
1,クルーズ,2138,情報・通信業,3017400,1160.000000
2,出前館,2484,情報・通信業,544400,439.000000
3,アップルインターナショナル,2788,卸売業,737400,423.000000
4,焼津水産化学工業,2812,食料品,566000,1215.000000
5,ポラリス・ホールディングス,3010,サービス業,652500,178.000000
6,クラウディアホールディングス,3607,繊維製品,569200,909.000000
7,ｅｎｉｓｈ,3667,情報・通信業,520600,308.000000
8,ジェイ・エスコムホールディングス,3779,情報・通信業,4802100,164.000000
9,サイバーステップ,3810,情報・通信業,1942500,451.000000


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
1,クルーズ,2138,情報・通信業,3017400,1160.000000
2,出前館,2484,情報・通信業,544400,439.000000
9,サイバーステップ,3810,情報・通信業,1942500,451.000000
10,リミックスポイント,3825,情報・通信業,1061100,239.000000
12,セプテーニ・ホールディングス,4293,サービス業,891100,379.000000
13,ミズホメディー,4595,医薬品,544300,2275.000000
15,守谷輸送機工業,6226,機械,950500,1277.000000
17,藤商事,6257,機械,718300,1407.000000
19,千代田化工建設,6366,建設業,1633200,384.000000
20,ユニバーサルエンターテインメント,6425,機械,1616900,2505.000000


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_grows_2

Not Found: No data found, symbol may be delisted
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,688700,1343.0
1,ウェッジホールディングス,2388,その他金融業,922800,146.0
2,ベースフード,2936,食料品,1058300,581.0
3,サンクゼール,2937,食料品,698500,3235.0
4,海帆,3133,小売業,1537700,477.0
5,スタジオアタオ,3550,小売業,734700,225.0
6,アプリックス,3727,情報・通信業,783600,168.0
7,Ａｉｍｉｎｇ,3911,情報・通信業,4525900,549.0
8,プレイド,4165,情報・通信業,3589100,718.0
9,ココナラ,4176,情報・通信業,885200,314.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
1,ウェッジホールディングス,2388,その他金融業,922800,146.0
3,サンクゼール,2937,食料品,698500,3235.0
4,海帆,3133,小売業,1537700,477.0
5,スタジオアタオ,3550,小売業,734700,225.0
8,プレイド,4165,情報・通信業,3589100,718.0
10,スパイダープラス,4192,情報・通信業,672000,763.0
11,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,658100,1039.0
18,Ｉ－ｎｅ,4933,化学,686500,3150.0
19,リプロセル,4978,化学,666300,271.0
20,ＡＮＹＣＯＬＯＲ,5032,情報・通信業,849900,3275.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')